# Transformations
## Abstract
- 本章では、点群に対する変換(平行移動、回転、スケーリング)を説明します。

## Introduction
点群に対して変換を用いることで、点群の回転や位置の移動を行うことが可能です。このsectionでは、以下の順で説明を行います。

- (1) 各変換についてコードメインの説明を行います。変換はsubsectionごとに、以下の順でまとめられています。
  - Translation
  - Rotation
  - Scaling
- (2) 上記変換を変換行列に沿って説明します。対応するsubscetion名は以下のとおりです。
  - Transformation matrix: 上記変換では扱わなかった応用例とともに、変換行列について説明します。


In [1]:
%load_ext autoreload
%autoreload 2

## Translation
本subsectionでは、点群に対するtranslationを紹介します。Translationは、ある一つのベクトルを点群に与え、点群に対してベクトル分の平行移動をさせる変換です。

本subsectionで利用するパッケージは以下の通りです。

In [5]:
# for translation
import numpy as np
from tutlibs.transformation import translation

# for description
from tutlibs.visualization import Points as visualizer
from tutlibs.io import Points as io
import inspect

Translationのコードは以下のとおりです。

In [6]:
# load a point cloud.
xyz, _, _ = io.read('../data/bunny_pc.ply')

# translation
vector = np.array([0.5, 0.05, 0.2])
t_xyz = translation(xyz, vector)

# visualization
color = np.tile(np.array([[1, 0, 0]]), (len(xyz), 1))
t_color = np.tile(np.array([[0, 1, 0]]), (len(t_xyz), 1))
visualizer.k3d(np.vstack([xyz, t_xyz]), colors=np.vstack([color, t_color]))

Output()

コードの出力では、赤色の点群がオリジナル`xyz`、緑の点群が平行移動後の点群`t_xyz`を示しています。コードでは、関数`translation`に一つのベクトル`vector`を与えることで、そのベクトルの方向に点群全体を移動させた後の点群`t_xyz`を取得しています。

次は`translation`の中身を出力し、どのようにして平行移動を行っているか確認します。確認は下のコード実行によって行います。


In [7]:
print(inspect.getsource(translation))

def translation(xyz:np.ndarray, vector:np.ndarray) -> np.ndarray:
    """Shift points.
    Args:
        xyz: (N, C)
        vector: (C)
    
    Return:
        translation xyz: (N, C)
    """
    return xyz + vector[np.newaxis, :]



上記の確認用の出力からも分かるように、点群中のすべての点の座標値`xyz`に対してベクトル`vecotr`を足すだけで平行移動後の点群は取得することが可能です(`np.newaxis`は点全体にベクトル値を足すために入れています)。


## Rotation
本subsectionでは、点群に対するrotationを紹介します。Rotationは点群をある中心点に基づいて回転させる変換です。このsubsectionの例では、原点を中心点として、中心点から伸びているX軸を回転軸として点群を90度回転させています。

本subsectionで利用するパッケージは以下の通りです。

In [8]:
# for Rotation
import numpy as np
from tutlibs.transformation import rotation

# for description
from tutlibs.visualization import Points as visualizer
from tutlibs.io import Points as io
import inspect

Rotationのコードは以下のとおりです。

In [9]:
# load a point cloud.
xyz, _, _ = io.read('../data/bunny_pc.ply')

# rotation
axis = 'x'
radian = np.pi / 2 # 90
t_xyz = rotation(xyz, axis, radian)

# visualization
color = np.tile(np.array([[1, 0, 0]]), (len(xyz), 1))
t_color = np.tile(np.array([[0, 1, 0]]), (len(t_xyz), 1))
visualizer.k3d(np.vstack([xyz, t_xyz]), colors=np.vstack([color, t_color]))

Output()

コードの出力では、赤色の点群がオリジナル`xyz`、緑の点群が回転後の点群`t_xyz`を示しています。コードでは、関数`rotation`に回転軸`axis`と回転角度`radian`を与えることで、その回転軸を軸として回転角度分だけ回った点群を取得することができます。

`rotation`の中身を出力し、どのようにして回転を行っているか確認します。確認は下のコード実行によって行います。

In [10]:
print(inspect.getsource(rotation))

def rotation(xyz:np.ndarray, axis:str, angle:float) -> np.ndarray:
    """Rotate points.

    Args:
        xyz: (N, C)
        axis: x, y or z
        angle: radian

    Return:
        rotated xyz: (N, C)
    """
    if axis == 'x':
        rotation_matrix = np.array([
            [1, 0, 0],
            [0, np.cos(angle), -np.sin(angle)],
            [0, np.sin(angle), np.cos(angle)]
        ])
    elif axis == 'y':
        rotation_matrix = np.array([
            [np.cos(angle), 0, np.sin(angle)],
            [0, 1, 0],
            [-np.sin(angle), 0, np.cos(angle)]
        ])
    elif axis == 'z':
        rotation_matrix = np.array([
            [np.cos(angle), -np.sin(angle), 0],
            [np.sin(angle), np.cos(angle), 0],
            [0, 0, 1]
        ])
    else:
        raise ValueError()

    rotation_xyz = np.matmul(rotation_matrix, xyz.T).T

    return rotation_xyz



回転された点群は、点群と回転行列の内積によって取得できます。上記では、回転行列は`rotation_matrix`に代入されており、代入される回転行列は軸によって変化します。この回転行列は、`xyz`との内積に使われ、内積の出力が回転された点群`rotation_xyz`として取得されます。

## Scaling
本subsectionでは、点群に対するscalingを紹介します。

本subsectionで利用するパッケージは以下の通りです。


In [1]:
# for Scaling
import numpy as np
from tutlibs.transformation import scaling

# for description
from tutlibs.visualization import Points as visualizer
from tutlibs.io import Points as io
import inspect

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2021-09-18 01:22:31,275 - utils - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 2021-09-18 01:22:31,276 - utils - NumExpr defaulting to 8 threads.


Rotationのコードは以下のとおりです。

In [3]:
# load a point cloud.
xyz, _, _ = io.read('../data/bunny_pc.ply')

# scaling
ratio = 0.5
t_xyz = scaling(xyz, ratio)

# visualization
color = np.tile(np.array([[1, 0, 0]]), (len(xyz), 1))
t_color = np.tile(np.array([[0, 1, 0]]), (len(t_xyz), 1))
visualizer.k3d(np.vstack([xyz, t_xyz]), colors=np.vstack([color, t_color]))

Output()

コードの出力では、赤色の点群がオリジナル`xyz`、緑の点群がスケーリング後の点群`t_xyz`を示しています。コードでは、関数`scaling`に

`rotation`の中身を出力し、どのようにして回転を行っているか確認します。確認は下のコード実行によって行います。

In [9]:
print(inspect.getsource(scaling))

def scaling(xyz:np.ndarray, ratio:float) -> np.ndarray:
    """Scale xyz.

    Args:
        xyz: (N, C)
        ratio: scaling ratio
    
    Return:
        scaled xyz: (N, C)
    """
    return xyz * ratio

